In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
from typing import Type, Optional, Union, Any, Tuple, Dict
import asyncio
from pathlib import Path
import pickle

import fbdev
from fbdev.component import func_component, PortSpecCollection, PortTypeSpec, PortSpec, PortType
from fbdev.graph import Graph, NodeSpec, EdgeSpec
from fbdev.node import Node, GraphComponentFactory
from fbdev.runtime import BatchExecutor
from fbdev.dev import enter_node_execution, save_graph_execution_cache

In [ ]:
from my_graph import g

In [ ]:
g.display_mermaid()

```mermaid
flowchart 
    classDef subgraph_zone fill:#111
    classDef loose_edge fill:#f00

    subgraph net__CHILD__add_one["add_one[]"]
        net__CHILD__add_one__PORTS__input__CHILD__a{{a}}
        net__CHILD__add_one__PORTS__output__CHILD__out([out])
    end
    subgraph net__CHILD__copier["copier[]"]
        net__CHILD__copier__PORTS__input__CHILD__a{{a}}
        net__CHILD__copier__PORTS__output__CHILD__out0([out0])
        net__CHILD__copier__PORTS__output__CHILD__out1([out1])
    end
    subgraph net__CHILD__printer["printer[]"]
        net__CHILD__printer__PORTS__input__CHILD__a{{a}}
    end
    subgraph net__CHILD__sender["sender[]"]
        net__CHILD__sender__PORTS__input__CHILD__a{{a}}
        net__CHILD__sender__PORTS__output__CHILD__out([out])
    end

        net__PORTS__input__CHILD__graph_in{{graph_in}}
        net__PORTS__output__CHILD__graph_out([graph_out])

    net__PORTS__input__CHILD__graph_in -.-> net__CHILD__add_one__PORTS__input__CHILD__a
    net__CHILD__add_one__PORTS__output__CHILD__out --> net__CHILD__copier__PORTS__input__CHILD__a
    net__CHILD__copier__PORTS__output__CHILD__out0 --> net__CHILD__printer__PORTS__input__CHILD__a
    net__CHILD__copier__PORTS__output__CHILD__out1 --> net__CHILD__sender__PORTS__input__CHILD__a
    net__CHILD__sender__PORTS__output__CHILD__out -.-> net__PORTS__output__CHILD__graph_out

```

Run the graph, and store the packet registry

In [ ]:
await save_graph_execution_cache(g)

In add_one
In copier
In printer1: 2
In sender


'Simulate' the execution of a node in the graph using the saved packet registry

In [ ]:
self = await enter_node_execution(graph=g, node_address='>net>copier')

Show what ports are available

In [ ]:
self.ports

PortType.INPUT:
  a:int
PortType.OUTPUT:
  out0:int
  out1:int
PortType.CONFIG:
PortType.SIGNAL:

Execution code

In [ ]:
packet = await self.ports.input.a.receive()
data = await self.consume_packet(packet)

await self.ports.output.out0.put(self.create_packet(data))
await self.ports.output.out1.put(self.create_packet(data))